In [13]:
import sys
sys.path.append('../../python_scripts')
import pymesh
import numpy as np
import logging, sys
logging.disable(sys.maxsize)
dirname = "/home/jxt/PPPM-TDBEM/dataset/bowl"
from fem import FEMmodel, Material, MatSet, LOBPCG_solver, SoundObj
from visulize import viewer
obj = SoundObj(dirname)

original mesh data
vertices:  (18370, 3)
faces:  (36736, 3)
bbox_min:  [-0.07172556 -0.03272797 -0.07172596]
bbox_max:  [0.07172556 0.02840117 0.0717259 ]
target cell_size:  0.0047817284295000005


In [14]:
obj.tetrahedralize()
obj.modal_analysis(k=200, material=Material(MatSet.Plastic))

tetrahedralized mesh data
vertices:  (3943, 3)
faces:  (7262, 3)
voxels:  (12190, 4)


In [15]:
obj.show_frequencys(num=20)
obj.visualize_modes(viewer, num=200)

[ 2087.3523  2092.634   4318.2935  4321.961   7173.7876  7181.415
  8086.8413  8723.094   8725.295  10174.988  10571.947  10652.587
 11847.941  11864.1045 13077.515  13097.714  13391.144  13437.459
 14370.775  14393.956 ]


Box(children=(Box(children=(IntSlider(value=0, description='Mode Index', layout=Layout(flex='3 1 auto'), max=1…

In [16]:
vertices = obj.tet_mesh.vertices
tets = obj.tet_mesh.voxels
print(vertices.shape)
print(tets.shape)
print(obj.eigenvalues.shape)
print(obj.eigenvectors.shape)
np.savetxt(dirname + "/vertices.txt", vertices)
np.savetxt(dirname + "/tets.txt", tets.astype(int))
np.savetxt(dirname + "/eigenvalues.txt", obj.eigenvalues)
np.savetxt(dirname + "/eigenvectors.txt", obj.eigenvectors)

(3943, 3)
(12190, 4)
(200,)
(11829, 200)


In [17]:
from physical import PhysicalAnimation
import pybullet as p
anim = PhysicalAnimation(dirname)
anim.generate()
anim.post_process_contacts()

Generating animation...


100%|██████████| 150000/150000 [00:12<00:00, 12499.73it/s]


post processing contact forces....


100%|██████████| 150000/150000 [00:09<00:00, 15027.97it/s]


In [22]:
SR = 44100
from tqdm import tqdm
import scipy.signal
from audio import save_audio, show_audio, IIR
sound_signal = np.zeros(int(anim.time_length * SR))
f = np.zeros((len(obj.eigenvalues), len(sound_signal)))
for contact in tqdm(anim.contact):
    t = int(contact[0] * SR)
    vertex_id = int(contact[1])
    force_direction = contact[2:5]
    impulse = contact[5] * (obj.eigenvectors[vertex_id*3, :] * force_direction[0] +
                             obj.eigenvectors[vertex_id*3+1, :] * force_direction[1] +
                             obj.eigenvectors[vertex_id*3+2, :] * force_direction[2])
    f[:, t] += impulse
# smooth force 
for mode_idx in range(200):
    sound_signal += IIR(f[mode_idx], obj.eigenvalues[mode_idx], obj.fem_model.material.alpha, obj.fem_model.material.beta, 1/SR)

# get acceleration
sound_signal = np.diff(sound_signal, n=2)
save_audio(sound_signal, SR, dirname + "/sound.wav")

100%|██████████| 207711/207711 [00:02<00:00, 90825.42it/s]


In [23]:
import os
from IPython.display import Video
video_name = dirname + '/animation.mp4'
audio_name = dirname + '/sound.wav'
output_name = dirname + '/animation+sound.mp4'
os.popen(f"ffmpeg -hide_banner -loglevel error -i {video_name} -i {audio_name} -c:v copy -c:a libvorbis -f mp4 {output_name} -y")
Video(output_name, embed=True)